<a href="https://colab.research.google.com/github/timothythampy/Big-Data-Techology/blob/main/PySpark_MovieRecommendation_Assignment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/nul
!wget -q https://mirrors.estointernet.in/apache/spark/spark-3.1.1/spark-3.1.1-bin-hadoop2.7.tgz
!tar xf spark-3.1.1-bin-hadoop2.7.tgz
!pip install -q findspark
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop2.7"
import findspark
findspark.init()
findspark.find()

'/content/spark-3.1.1-bin-hadoop2.7'

In [ ]:
! pip install -q kaggle

In [ ]:
from google.colab import files
files.upload()

Saving archive.zip to archive.zip


In [ ]:
! mkdir ~/.kaggle
! cp kaggle.json ~/.kaggle/
! chmod 600 ~/.kaggle/kaggle.json
! kaggle datasets list

In [ ]:
!kaggle datasets download -d grouplens/movielens-20m-dataset

Traceback (most recent call last):
  File "/usr/local/bin/kaggle", line 5, in <module>
    from kaggle.cli import main
  File "/usr/local/lib/python2.7/dist-packages/kaggle/__init__.py", line 23, in <module>
    api.authenticate()
  File "/usr/local/lib/python2.7/dist-packages/kaggle/api/kaggle_api_extended.py", line 146, in authenticate
    self.config_file, self.config_dir))
IOError: Could not find kaggle.json. Make sure it's located in /root/.kaggle. Or use the environment method.


In [ ]:
!unzip /content/archive.zip

Archive:  /content/archive.zip
  inflating: genome_scores.csv       
  inflating: genome_tags.csv         
  inflating: link.csv                
  inflating: movie.csv               
  inflating: rating.csv              
  inflating: tag.csv                 


In [ ]:
import os
import sys
os.environ['PYSPARK_PYTHON'] = sys.executable
os.environ['PYSPARK_DRIVER_PYTHON'] = sys.executable

In [ ]:
def get_mat_sparsity(ratings):
    # Count the total number of ratings in the dataset
    count_nonzero = ratings.select("rating").count()

    # Count the number of distinct userIds and distinct movieIds
    total_elements = ratings.select("userId").distinct().count() * ratings.select("movieId").distinct().count()

    # Divide the numerator by the denominator
    sparsity = (1.0 - (count_nonzero *1.0)/total_elements)*100
    print("The ratings dataframe is ", "%.2f" % sparsity + "% sparse.")

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('Recommendations').getOrCreate()
movies = spark.read.csv("movie.csv",header=True)
ratings = spark.read.csv("rating.csv",header=True)
ratings.show()

+------+-------+------+-------------------+
|userId|movieId|rating|          timestamp|
+------+-------+------+-------------------+
|     1|      2|   3.5|2005-04-02 23:53:47|
|     1|     29|   3.5|2005-04-02 23:31:16|
|     1|     32|   3.5|2005-04-02 23:33:39|
|     1|     47|   3.5|2005-04-02 23:32:07|
|     1|     50|   3.5|2005-04-02 23:29:40|
|     1|    112|   3.5|2004-09-10 03:09:00|
|     1|    151|     4|2004-09-10 03:08:54|
|     1|    223|     4|2005-04-02 23:46:13|
|     1|    253|     4|2005-04-02 23:35:40|
|     1|    260|     4|2005-04-02 23:33:46|
|     1|    293|     4|2005-04-02 23:31:43|
|     1|    296|     4|2005-04-02 23:32:47|
|     1|    318|     4|2005-04-02 23:33:18|
|     1|    337|   3.5|2004-09-10 03:08:29|
|     1|    367|   3.5|2005-04-02 23:53:00|
|     1|    541|     4|2005-04-02 23:30:03|
|     1|    589|   3.5|2005-04-02 23:45:57|
|     1|    593|   3.5|2005-04-02 23:31:01|
|     1|    653|     3|2004-09-10 03:08:11|
|     1|    919|   3.5|2004-09-1

In [ ]:
movies.show()

+-------+--------------------+--------------------+
|movieId|               title|              genres|
+-------+--------------------+--------------------+
|      1|    Toy Story (1995)|Adventure|Animati...|
|      2|      Jumanji (1995)|Adventure|Childre...|
|      3|Grumpier Old Men ...|      Comedy|Romance|
|      4|Waiting to Exhale...|Comedy|Drama|Romance|
|      5|Father of the Bri...|              Comedy|
|      6|         Heat (1995)|Action|Crime|Thri...|
|      7|      Sabrina (1995)|      Comedy|Romance|
|      8| Tom and Huck (1995)|  Adventure|Children|
|      9| Sudden Death (1995)|              Action|
|     10|    GoldenEye (1995)|Action|Adventure|...|
|     11|American Presiden...|Comedy|Drama|Romance|
|     12|Dracula: Dead and...|       Comedy|Horror|
|     13|        Balto (1995)|Adventure|Animati...|
|     14|        Nixon (1995)|               Drama|
|     15|Cutthroat Island ...|Action|Adventure|...|
|     16|       Casino (1995)|         Crime|Drama|
|     17|Sen

In [ ]:
movie_ratings = ratings.join(movies, ['movieId'], 'left')

In [ ]:
get_mat_sparsity(ratings)

The ratings dataframe is  99.46% sparse.


In [ ]:
(train, test) = ratings.randomSplit([0.8, 0.2], seed = 1234)

In [ ]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.tuning import ParamGridBuilder, CrossValidator

In [ ]:
als = ALS(
            userCol = 'userId',
          itemCol = 'movieId',
          ratingCol = 'rating',
          nonnegative = True,
          implicitPrefs = False,
          coldStartStrategy = 'drop'

)

In [ ]:
type(als)

pyspark.ml.recommendation.ALS

In [ ]:
param_grid = ParamGridBuilder()\
              .addGrid(als.rank,[50])\
              .addGrid(als.regParam,[.15])\
              .build()



evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating', predictionCol= 'prediction')

print(f'Total models = {len(param_grid)}')

Total models = 1


In [ ]:
cv = CrossValidator(estimator=als,
                    estimatorParamMaps=param_grid,
                    evaluator = evaluator,
                    numFolds=5)

In [ ]:
train.dtypes

[('userId', 'string'),
 ('movieId', 'string'),
 ('rating', 'string'),
 ('timestamp', 'string')]

In [ ]:
from pyspark.sql.types import DoubleType
train = train.withColumn("userId", train.userId.cast(DoubleType()))
train = train.withColumn("movieId", train.movieId.cast(DoubleType()))
train = train.withColumn("rating", train.rating.cast(DoubleType()))

In [ ]:
model = cv.fit(train)
best_model = model.bestModel

In [ ]:
# Print best_model
print(type(best_model))

# Complete the code below to extract the ALS model parameters
print("**Best Model**")

print("  Rank:", best_model._java_obj.parent().getRank())

print("  MaxIter:", best_model._java_obj.parent().getMaxIter())
print("  RegParam:", best_model._java_obj.parent().getRegParam())

<class 'pyspark.ml.recommendation.ALSModel'>
**Best Model**
  Rank: 50
  MaxIter: 10
  RegParam: 0.15


In [ ]:
test = test.withColumn("userId", test.userId.cast(DoubleType()))
test = test.withColumn("movieId", test.movieId.cast(DoubleType()))
test = test.withColumn("rating", test.rating.cast(DoubleType()))

In [ ]:
test_predictions = best_model.transform(test)
RMSE = evaluator.evaluate(test_predictions)
print(RMSE)

0.8402204760199977


In [ ]:
test_predictions.show()

+--------+-------+------+-------------------+----------+
|  userId|movieId|rating|          timestamp|prediction|
+--------+-------+------+-------------------+----------+
| 74757.0|  148.0|   3.5|2003-09-29 16:35:35| 2.6596174|
| 96393.0|  148.0|   3.0|2000-09-28 19:41:30| 2.6801524|
| 88527.0|  148.0|   2.0|2000-08-07 14:48:44|  2.405804|
| 22884.0|  148.0|   3.0|1999-12-11 21:31:08| 2.6750886|
| 80168.0|  148.0|   4.0|1996-06-26 20:16:30| 2.8886373|
| 46944.0|  148.0|   2.0|1996-08-13 19:40:14| 2.7424438|
| 54726.0|  148.0|   5.0|1996-05-21 18:34:30| 3.2116697|
| 90757.0|  148.0|   3.0|2000-09-26 20:36:10| 3.1595242|
|125969.0|  148.0|   3.0|1999-10-17 02:57:17| 2.9611697|
| 81300.0|  148.0|   1.0|2001-02-25 16:36:04| 2.8208444|
| 78276.0|  148.0|   2.0|1999-08-21 23:22:54| 2.8967924|
| 61663.0|  148.0|   2.0|1997-09-18 10:11:52| 3.1462493|
|132268.0|  148.0|   2.0|1999-01-03 23:28:53| 2.2691367|
|  4914.0|  148.0|   2.0|2000-12-07 20:12:34| 2.6950867|
| 37331.0|  148.0|   3.0|1999-1

In [ ]:
recommendations = best_model.recommendForAllUsers(5)
recommendations.show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[{126219, 8.41106...|
|   463|[{126219, 8.12306...|
|   471|[{126219, 7.52317...|
|   496|[{126219, 7.76550...|
|   833|[{126219, 6.67776...|
|  1088|[{126219, 6.44078...|
|  1238|[{126219, 7.51580...|
|  1342|[{126219, 8.05850...|
|  1580|[{126219, 5.49926...|
|  1591|[{126219, 8.35903...|
|  1645|[{126219, 7.82805...|
|  1829|[{126219, 8.26063...|
|  1959|[{126219, 6.00974...|
|  2122|[{126219, 5.43473...|
|  2142|[{126219, 8.01725...|
|  2366|[{126219, 7.72466...|
|  2659|[{126219, 8.00457...|
|  2866|[{126219, 7.21808...|
|  3175|[{126219, 8.42237...|
|  3749|[{126219, 7.59328...|
+------+--------------------+
only showing top 20 rows



In [ ]:
from pyspark.sql.functions import split, explode
from pyspark.sql.functions import col

nrecommendations = recommendations\
.withColumn("rec_exp", explode("recommendations"))\
    .select('userId', col("rec_exp.movieId"), col("rec_exp.rating"))
recommendations.limit(10).show()

+------+--------------------+
|userId|     recommendations|
+------+--------------------+
|   148|[{126219, 8.41106...|
|   463|[{126219, 8.12306...|
|   471|[{126219, 7.52317...|
|   496|[{126219, 7.76550...|
|   833|[{126219, 6.67776...|
|  1088|[{126219, 6.44078...|
|  1238|[{126219, 7.51580...|
|  1342|[{126219, 8.05850...|
|  1580|[{126219, 5.49926...|
|  1591|[{126219, 8.35903...|
+------+--------------------+



In [ ]:
nrecommendations.join(movies, on='movieId').filter('userId = 100').show()

+-------+------+---------+--------------------+-----------------+
|movieId|userId|   rating|               title|           genres|
+-------+------+---------+--------------------+-----------------+
| 126219|   100| 6.516889|    Marihuana (1936)|Documentary|Drama|
| 121029|   100| 5.355795|No Distance Left ...|      Documentary|
|  98595|   100|5.2942824|Peppermint Soda (...|     Comedy|Drama|
|  81117|   100|5.1863623|Moth, The (Cma) (...|            Drama|
| 112423|   100|5.0537143|I Belong (Som du ...|            Drama|
+-------+------+---------+--------------------+-----------------+



In [ ]:
ratings.join(movies, on='movieId').filter('userId = 100').sort('rating', ascending=False).limit(10).show()

+-------+------+------+-------------------+--------------------+--------------------+
|movieId|userId|rating|          timestamp|               title|              genres|
+-------+------+------+-------------------+--------------------+--------------------+
|    680|   100|     5|1996-06-25 16:58:31|Alphaville (Alpha...|Drama|Mystery|Rom...|
|   1449|   100|     5|1997-06-09 16:38:17|Waiting for Guffm...|              Comedy|
|     50|   100|     5|1996-06-25 16:24:49|Usual Suspects, T...|Crime|Mystery|Thr...|
|    293|   100|     5|1996-06-25 16:28:27|Léon: The Profess...|Action|Crime|Dram...|
|    235|   100|     4|1996-06-25 16:28:27|      Ed Wood (1994)|        Comedy|Drama|
|    260|   100|     4|1997-06-09 16:40:56|Star Wars: Episod...|Action|Adventure|...|
|    162|   100|     4|1996-06-25 16:43:19|        Crumb (1994)|         Documentary|
|    265|   100|     4|1996-06-25 16:29:49|Like Water for Ch...|Drama|Fantasy|Rom...|
|    288|   100|     4|1996-06-25 16:24:07|Natural Bor